In [4]:


# Guidelines :
# 1. Convert every string to lower-case
# 2. Remove any punctuation, i.e., any character other than [a...z]
# 3. For each sub-question submit 2 files: 
# text file with the results (any format you believe is more suitable) and the python code


# http://spark.apache.org/docs/latest/programming-guide.html 
# https://spark.apache.org/docs/1.1.1/api/python/pyspark.rdd.RDD-class.html#mapValues

In [3]:
# a) For each word in the input text, you must compute the occurrences of other words appear in every line that it appeared. 
# For example, assume that we have the following line as an input text.
# “Theseus seemed to me to resemble Romulus in many particulars.”
# With the word “Theseus”: “to” occurs twice, “seemed” occurs once, “me” occurs once and so on.
# With the word “to”, “Theseus” occurs once, “to” occurs once (not zero times!), and so on.
# For this assignment, you are going to compute the occurrences of every word for all lines, not just for a single line.


txt = sc.textFile("/Users/nanazhu/Documents/LosAlamos/BigDataComputing/Spark/pg674.txt")
# txt = sc.textFile("/Users/nanazhu/Documents/LosAlamos/BigDataComputing/Spark/pg.txt")
import string

def func_map(line):
    line = line.translate(str.maketrans( '', '', string.punctuation)).lower().split()
    list_pairs=[]    
    list_unique=[]
    for i in range(len(line)) : 
        key_a =line[i]
        if key_a in list_unique:
            continue
        else:
            list_unique.append(key_a)
        for j in range(len(line)):
            if j == i:
                continue
            key_b = line[j]
            list_pairs.append(((key_a,key_b),1))
    return list_pairs
    
txtmap = txt.flatMap(func_map)
# txtmap.collect()

txtreduce = txtmap.reduceByKey(lambda a, b: a + b)
# txtreduce.collect()

txtmap_map = txtreduce.map(lambda line: (line[0][0], (line[0][1],line[1]))).sortByKey()
# txtmap_map.collect()
txtreduce_reduce = txtmap_map.groupByKey().mapValues(list).sortByKey()
# txtreduce_reduce.mapValues(list).collect()
txtreduce_reduce.saveAsTextFile("/Users/nanazhu/Documents/LosAlamos/BigDataComputing/Spark/result_A")


In [6]:
# b) Produce a list of words, with the top-10 words that appear with this word for all lines.

txt = sc.textFile("/Users/nanazhu/Documents/LosAlamos/BigDataComputing/Spark/pg674.txt")
# txt = sc.textFile("/Users/nanazhu/Documents/LosAlamos/BigDataComputing/Spark/pg.txt")
import string

def func_map(line):
    line = line.translate(str.maketrans( '', '', string.punctuation)).lower().split()
    list_pairs=[]    
    list_unique=[]
    for i in range(len(line)) : 
        key_a =line[i]
        if key_a in list_unique:
            continue
        else:
            list_unique.append(key_a)
        for j in range(len(line)):
            if j == i:
                continue
            key_b = line[j]
            list_pairs.append(((key_a,key_b),1))
    return list_pairs
    
txtmap = txt.flatMap(func_map)
# txtmap.collect()

txtreduce = txtmap.reduceByKey(lambda a, b: a + b)
# txtreduce.collect()

txtmap_map = txtreduce.map(lambda line: (line[0][0], (line[0][1],line[1])))
# txtmap_map.collect()

def sort_list(x):
    top_k = 3
    return sorted(x, key = lambda item: item[1],reverse=True)[0:top_k]
txtreduce_reduce = txtmap_map.groupByKey().mapValues(sort_list).sortByKey()
txtreduce_reduce.mapValues(list).collect()
txtreduce_reduce.saveAsTextFile("/Users/nanazhu/Documents/LosAlamos/BigDataComputing/Spark/result_B")

In [7]:
# c) For each word in the input text, you must compute the occurrences of other words that appear in every line that it appeared, 
# right after the appearance of the word. For example, assume that we have the following line as an input text
# “Theseus seemed to me to resemble Romulus in many particulars.”
# With the word “Theseus”, “seemed” occurs once. With the word “seemed”, “to” occurs once.
# Produce a list of words, with the top-10 words that appear right after this word for all lines.


txt = sc.textFile("/Users/nanazhu/Documents/LosAlamos/BigDataComputing/Spark/pg674.txt")
# txt = sc.textFile("/Users/nanazhu/Documents/LosAlamos/BigDataComputing/Spark/pg.txt")
import string

def func_map(line):
    line = line.translate(str.maketrans( '', '', string.punctuation)).lower().split()
    list_pairs=[]  
    for i in range(len(line)) :
        if i +1 == len(line):
            break
        j = i + 1
        key_a =line[i]
        key_b = line[j]
        list_pairs.append(((key_a,key_b),1))
    return list_pairs
    
txtmap = txt.flatMap(func_map)
# txtmap.collect()

txtreduce = txtmap.reduceByKey(lambda a, b: a + b)
# txtreduce.collect()

txtmap_map = txtreduce.map(lambda line: (line[0][0], (line[0][1],line[1])))
# txtmap_map.collect()

def sort_list(x):
    top_k = 10
    return sorted(x, key = lambda item: item[1],reverse=True)[0:top_k]
txtreduce_reduce = txtmap_map.groupByKey().mapValues(sort_list).sortByKey()
# txtreduce_reduce.mapValues(list).collect()
txtreduce_reduce.saveAsTextFile("/Users/nanazhu/Documents/LosAlamos/BigDataComputing/Spark/result_C")

In [8]:
# d) Generalize step (c) so that you include all words that appear up to distance k after a given word.

txt = sc.textFile("/Users/nanazhu/Documents/LosAlamos/BigDataComputing/Spark/pg674.txt")
# txt = sc.textFile("/Users/nanazhu/Documents/LosAlamos/BigDataComputing/Spark/pg.txt")
import string

def func_map(line):
    line = line.translate(str.maketrans( '', '', string.punctuation)).lower().split()
    list_pairs=[]
    k = 3
    for i in range(len(line)) :
        if i+k > len(line):
            end =  len(line)
        else:
            end = i+k
        key_a =line[i]
        for j in range(i +1,end):
            key_b = line[j]
            list_pairs.append(((key_a,key_b),1))
    return list_pairs
    
txtmap = txt.flatMap(func_map).sortByKey()
# txtmap.collect()

txtreduce = txtmap.reduceByKey(lambda a, b: a + b)
# txtreduce.collect()

txtmap_map = txtreduce.map(lambda line: (line[0][0], (line[0][1],line[1])))
# txtmap_map.collect()

def sort_list(x):
    top_k = 10
    return sorted(x, key = lambda item: item[1],reverse=True)[0:top_k]
txtreduce_reduce = txtmap_map.groupByKey().mapValues(sort_list).sortByKey()
txtreduce_reduce.mapValues(list).collect()
txtreduce_reduce.saveAsTextFile("/Users/nanazhu/Documents/LosAlamos/BigDataComputing/Spark/result_D")
